# 使用情境 - 1

你現在是一個作文批改老師，你想要使用所學的 Langchain 知識來幫助你批改作文，建立一個作文批改的模型，輸入一篇作文，自訂輸出格式以滿足 cell 內題目的需求。

You should how to:

    1. define schema
    2. build output parser
    3. define input templetes
    4. chain them together
    5. feed your model with an article

- 範例如下：
    - 輸入：
        ```python
        input_ = {
            "article": "愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。"
        }
        ```
    - 輸出
        ```python
        {
            "標題": "穿越你心中夏豔的森林",
            "分數": 6,
            "評語": "這篇文章的詩意和意境很美，但是在表達上有些不清晰。建議在文章中加入更多具體的描述和情感表達，讓讀者更能感受到你想傳達的意思。",
            "改寫": "穿越你心中夏豔的森林\n樹葉翻飛，低語著秘密\n你的心溫暖而明亮\n讓人願意放下煩憂，踏上旅程\n我選了一片葉子，寫下你的名字\n當時間匆匆流逝\n仍有風逆行回來\n我一遍一遍朗讀\n感受著你的存在。",
        }
        ```

In [1]:
import warnings
from pprint import pprint
from langchain.prompts import ChatPromptTemplate
from langchain.chains.openai_functions.base import convert_to_openai_function
from langchain.output_parsers import PydanticOutputParser
from langchain.output_parsers.openai_functions import PydanticOutputFunctionsParser
from langchain.pydantic_v1 import BaseModel, Field, root_validator
from langchain_setup import ChatOpenAI

In [5]:
"""
======================================================
cousre practice: 
用課程先前所學，建立一個LLM模型來批改文章，並以特定框架output

給定一篇文章 `article`，讓模型給出以下四個分數：

1. 依照切題程度給予評分，分數須介於 1~10分之間
2. 依照內容給予評分，分數須介於 1~10分之間
3. 依照創意程度給予評分，分數須介於 1~10分之間
4. 依照通順程度給予評分，分數須介於 1~10分之間

===================================================
"""

input_ = {
    "article": "愛迪生說：「友誼能增進快樂，減少痛苦」，可見朋友有多重要。朋友像一根拐杖，當我跌倒的時候，扶我起來，支撐我繼續往前走；朋友像一位溫柔的老師，當我功課不會寫時，他會很用心的教導我；朋友像一顆開心果，當我難過或傷心時，講笑話給我聽；朋友像一個太陽，當我遇到困難時，覺得人生黑暗的時候，他會溫暖我的心。猶記一年級的時候，我有一位很要好的朋友，她喜歡運動和打球，而我也很喜歡運動，下課的時候，我常常跟她一起玩。她很大方，也很有禮貌，又喜歡幫老師做事，人緣好，所以大家都選她當班長。在她身上我看到很多優點，也學到很多，有了她的陪伴，所以每天在學校都過得很開心。有一次我在操場上跌倒流血，倒在地上爬不起來，是她扶我起來，陪我到健康中心。接下來那幾天，她幫我打掃、交作業，這讓我深深體會好朋友是多麼重要。雖然我們升上中年級後沒有在同一班，但我們還是思念著對方，下課時常常聚在一起，我的心中仍將她視為最好的朋友。如果一個人沒有朋友，就找不到陪她一起玩樂和訴苦的人，這樣就會很孤單。朋友是一生中絕對不能少的，缺少了他，我可能每天心情不好而感到孤獨寂寞。有了好朋友，是一件很幸福的事，也要懂得珍惜，建立的友誼才能長久。"
}


# ======================================================
# Define schema,validation and postprocessing
# ======================================================
class Article(BaseModel):
    """批改作文的老師"""

    relevance: int = Field(description="這篇文章的內容與題目的相關程度。介於 1 到 10。")
    writing: int = Field(description="這篇文章的文字遣詞是否精練，以及其內容深度。介於 1 到 10。")
    creativity: int = Field(description="這篇文章發想內容的創意程度。介於 1 到 10。")
    coherence: int = Field(description="文章的通順程度。介於 1 到 10。")

    @root_validator()
    def validate_values(cls, values: dict) -> dict:
        for key in values.keys():
            if not (1 <= values[key] <= 10):
                warnings.warn(f"分數 {values[key]} 不在 [1,10] 內，自動調整為 1 或 10")
                values[key] = max(min(1, values[key]), 10)
        return values


# build ouput parser
output_parser = PydanticOutputFunctionsParser(pydantic_schema=Article)

# ======================================================
# Define input template
# ======================================================
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一名國文教師，你現在要批改學生的作文，題目為朋友，評分標準需為以下四項：切題、文筆、創意、通順。"),
        ("human", "你好，請問可以批改我的作文嗎。"),
        ("ai", "好的，請給我看你的文章。"),
        ("human", "這是我的文章 :\n\n{article}。"),
    ],
)

model = ChatOpenAI(temperature=0)

function_kwargs = convert_to_openai_function(Article)

openaifn_kwargs = {"functions": [function_kwargs], "function_call": {"name": "Article"}}

chain = prompt_template | model.bind(**openaifn_kwargs) | output_parser

output = chain.invoke(input_)
print(
    {
        "切題程度": output.relevance,
        "文筆": output.writing,
        "創意": output.creativity,
        "通順": output.coherence,
    }
)

{'切題程度': 8, '文筆': 7, '創意': 6, '通順': 9}


# 使用情境 - 2

請同學利用情境分支 - RunnableBranch來處理兩種不同種類的問題，並建立兩個語言模型來回答：
    
  - 第一個種類的為問題內包含「景點」的問句，建立一個chain來回答使用者的問題並推薦景點
  - 第二個種類的為問題內包含「美食」的問句，建立另一個chain來回答使用者的問題並推薦食物
        
    
  - 範例如下：
    
            輸入：台灣有什麼好吃的美食嗎
            輸出：
                Scenario Branch Output:
                ('臭豆腐', '臭豆腐是台灣非常受歡迎的特色小吃之一。它是由豆腐經過發酵製成，因此帶有一種獨特的臭味。雖然聞起來可能讓人有些驚訝，但臭豆腐的味道卻是令人著迷的。它通常會先炸至外酥內軟，再搭配醬料，如醬油、辣椒醬等，增添風味。臭豆腐酥脆外皮與軟嫩內餡的組合，讓人一試難忘。')
                
            輸入：台灣有什麼好玩的景點嗎
            輸出：
               Scenario Branch Output:
                {'景點': '九份老街', '介紹': '九份老街是一個保存完整的日本式街道，位於台灣基隆市內門里，是一個歷史悠久的地區。這條街道上有許多傳統的小店，販賣著各式各樣的特色商品和美食。遊客可以在這裡品嚐到台灣傳統的小吃，如芋圓、鳳梨酥和茶葉蛋，也可以購買紀念品和手工藝品。此外，九份老街的風景也非常迷人，遊客可以欣賞到壯麗的山景和海景，還可以漫步在狹窄的巷弄中，感受悠閒的氛圍。'}
    

In [51]:
from langchain.prompts import ChatPromptTemplate
from langchain_setup import ChatOpenAI
from langchain.output_parsers import PydanticOutputParser
from langchain.schema.runnable import (
    RunnableLambda,
    RunnableMap,
    RunnableBranch,
    RunnablePassthrough,
)

# Scenario: Developing a Chatbot Pipeline

# Components Initialization
# Prompt
attraction_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位智能導遊機器人。請根據輸入的疑問滿足旅客的要求。"),
        ("human", "請告訴我{place}有什麼好玩的，並推薦當地的一個景點並向我介紹"),
    ],
)

# attraction_model
attraction_model = ChatOpenAI(temperature=0)


class Attraction_descriptor(BaseModel):
    """介紹景點的導遊"""

    attraction: str = Field(description="推薦的旅遊景點")
    introduction: str = Field(description="景點的簡要介紹，介紹該景點的特色")


# build ouput parser
attraction_output_parser = PydanticOutputFunctionsParser(
    pydantic_schema=Attraction_descriptor
)

attraction_function_kwargs = convert_to_openai_function(Attraction_descriptor)
attraction_openaifn_kwargs = {
    "functions": [attraction_function_kwargs],
    "function_call": {"name": "Attraction_descriptor"},
}

# 1. Constructing the Basic Chain - Prompt | Model | Output Parser
attraction_chain = (
    attraction_prompt
    | attraction_model.bind(**attraction_openaifn_kwargs)
    | attraction_output_parser
)

# Creating RunnableLambda instances for the functions

food_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "你是一位智能導遊機器人。請根據輸入的疑問滿足旅客的要求。"),
        ("human", "請告訴我{place}有什麼特色美食，簡要的介紹一個特色美食給我"),
    ],
)

# food_model
food_model = ChatOpenAI(temperature=0)


class food_descriptor(BaseModel):
    """介紹食物的導遊"""

    food: str = Field(description="美食名稱")
    introduction: str = Field(description="美食的簡要介紹")


# build ouput parser
food_output_parser = PydanticOutputFunctionsParser(pydantic_schema=food_descriptor)


food_function_kwargs = convert_to_openai_function(food_descriptor)
food_openaifn_kwargs = {
    "functions": [food_function_kwargs],
    "function_call": {"name": "food_descriptor"},
}
# Exercise:

# 3. Combining Runnable Functions using RunnableLambda
food_chain = food_prompt | food_model.bind(**food_openaifn_kwargs) | food_output_parser

# 5. Handling Multiple Scenarios - RunnableBranch


def route(question: str):
    if "景點" in question:
        return attraction_chain.invoke({"place": question})
    elif "美食" in question:
        return food_chain.invoke({"place": question})
    else:
        return "很抱歉，我不太明白您的需求。"


route_runnable = RunnableLambda(route)

# 6. Invoking the Scenario Branch
scenario_input = "台灣有什麼好吃的美食嗎?"
scenario_output = route_runnable.invoke(scenario_input)
print("\nScenario Branch Output:")
print(scenario_output)


Scenario Branch Output:
food='小籠包' introduction='小籠包是台灣非常有名的特色美食之一。它是一種以豬肉為餡料，用麵粉皮包裹而成的小型餃子。小籠包的特色在於其餡料鮮嫩多汁，皮薄餡多，一口咬下去可以感受到豐富的湯汁和鮮美的肉香。最著名的小籠包來自台北的鼎泰豐餐廳，是許多遊客必訪的美食之一。'
